In [1]:
!git init .
!git remote add origin https://github.com/kashparty/STP-GSR.git
!git pull origin discriminator

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 127 (delta 10), reused 8 (delta 4), pack-reused 102 (from 1)
Receiving objects: 100% (127/127), 64.80 MiB | 3.98 MiB/s, done.
Resolving deltas: 100% (53/53), done.
From https://github.com/kashparty/STP-GSR
 * branch            discriminator -> FETCH_HEAD
 * [new branch]      discriminator -> origin/discriminator
U

In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
import os
import hydra
import torch
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import KFold
from hydra import compose, initialize

from src.train import train, eval
from src.plot_utils import plot_adj_matrices
from src.dataset import load_dataset

def main():
    with initialize(version_base=None, config_path="configs"):
        config = compose(config_name="experiment")

    torch.cuda.empty_cache()

    if torch.cuda.is_available():
        print("Running on GPU")
    else:
        print("Running on CPU")

    kf = KFold(n_splits=config.experiment.kfold.n_splits,
               shuffle=config.experiment.kfold.shuffle,
               random_state=config.experiment.kfold.random_state)

    # Initialize folder structure for this run
    base_dir = config.experiment.base_dir
    model_name = config.model.name
    dataset_type = config.dataset.name
    run_name = config.experiment.run_name
    run_dir = f'{base_dir}/{model_name}/{dataset_type}/{run_name}/'

    # Load dataset
    source_data, target_data = load_dataset(config)

    for fold, (train_idx, val_idx) in enumerate(kf.split(source_data)):
        print(f"Training Fold {fold+1}/3")

        # Initialize results directory
        res_dir = f'{run_dir}fold_{fold+1}/'
        if not os.path.exists(res_dir):
            os.makedirs(res_dir)

        # Fetch training and val data for this fold
        source_data_train = [source_data[i] for i in train_idx]
        target_data_train = [target_data[i] for i in train_idx]
        source_data_val = [source_data[i] for i in val_idx]
        target_data_val = [target_data[i] for i in val_idx]

        # Train model for this fold
        train_output = train(config,
                              source_data_train,
                              target_data_train,
                              source_data_val,
                              target_data_val,
                              res_dir)

        # Evaluate model for this fold
        eval_output, eval_loss = eval(config,
                                      train_output['model'],
                                      source_data_val,
                                      target_data_val,
                                      train_output['critereon'])

        # Final evaluation loss for this fold
        print(f"Final Validation Loss (Target): {eval_loss}")

        # Save source, taregt, and eval output for this fold
        np.save(f'{res_dir}/eval_output.npy', np.array(eval_output))
        np.save(f'{res_dir}/source.npy', np.array([s['mat'] for s in source_data_val]))
        np.save(f'{res_dir}/target.npy', np.array([t['mat'] for t in target_data_val]))


        # Plot predictions for a random sample
        idx = 6
        source_mat_test = source_data_val[idx]['mat']
        target_mat_test = target_data_val[idx]['mat']
        eval_output_t = eval_output[idx]

        plot_adj_matrices(source_mat_test,
                          target_mat_test,
                          eval_output_t,
                          idx,
                          res_dir,
                          file_name=f'eval_sample{idx}')


main()

Running on CPU
Training Fold 1/3


  0%|          | 0/111 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (268x268 and 320x160)

In [1]:
from hydra import compose, initialize
from src.models.stp_gsr import STPGSR
import torch
from tqdm import tqdm

with initialize(version_base=None, config_path="configs"):
    config = compose(config_name="experiment")

model = STPGSR(config)
model.load_state_dict(torch.load("results/stp_gsr/train/run4/fold_3/model.pth", map_location=torch.device("cuda")))

/vol/bitbucket/akm20/conda/envs/DGLCW2/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/vol/bitbucket/akm20/conda/envs/DGLCW2/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


<All keys matched successfully>

In [2]:
from src.matrix_vectorizer import MatrixVectorizer
from src.dataset import create_pyg_graph
from functools import partial
import numpy as np

source_vectorized = np.genfromtxt("lr_test.csv", delimiter=",", skip_header=1)
source_mat_all = [MatrixVectorizer.anti_vectorize(A, 160) for A in source_vectorized]

source_mat_all = [torch.tensor(x, dtype=torch.float) for x in source_mat_all]
pyg_partial = partial(create_pyg_graph, node_feature_init="adj", node_feat_dim=160)

source_pyg_all = [pyg_partial(x, 160) for x in source_mat_all]
source_data = [{'pyg': source_pyg, 'mat': source_mat} for source_pyg, source_mat in zip(source_pyg_all, source_mat_all)]

In [3]:
from src.dual_graph_utils import revert_dual

model.eval()
eval_output = []

with torch.no_grad():
    for source in tqdm(source_data):
        source_g = source['pyg']

        model_pred, model_target = model(source_g, None)
        pred_m = revert_dual(model_pred, 268)    # (n_t, n_t)
        pred_m = pred_m.cpu().numpy()
        eval_output.append(pred_m)

eval_output

100%|██████████| 112/112 [03:48<00:00,  2.04s/it]


[array([[0.        , 0.52883095, 0.49526468, ..., 0.        , 0.        ,
         0.        ],
        [0.52883095, 0.        , 0.69366795, ..., 0.        , 0.07750527,
         0.        ],
        [0.49526468, 0.69366795, 0.        , ..., 0.        , 0.01696083,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.3802377 ,
         0.27525485],
        [0.        , 0.07750527, 0.01696083, ..., 0.3802377 , 0.        ,
         0.3613272 ],
        [0.        , 0.        , 0.        , ..., 0.27525485, 0.3613272 ,
         0.        ]], dtype=float32),
 array([[0.        , 0.47147387, 0.5048311 , ..., 0.        , 0.02349842,
         0.        ],
        [0.47147387, 0.        , 0.4934655 , ..., 0.        , 0.20071074,
         0.16617821],
        [0.5048311 , 0.4934655 , 0.        , ..., 0.        , 0.143134  ,
         0.10236943],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.29490924,
         0.2521445 

In [4]:
from src.matrix_vectorizer import MatrixVectorizer
import pandas as pd

test_array = np.concatenate([MatrixVectorizer.vectorize(eo) for eo in eval_output])

output_df = pd.DataFrame({"Predicted": test_array.flatten()})
output_df.index = np.arange(1, len(output_df) + 1)
output_df.to_csv("submission.csv", index_label="ID")
output_df

,Predicted
1,0.528831
2,0.495265
3,0.693668
4,0.486140
5,0.321900
...,...
4007132,0.000000
4007133,0.000000
4007134,0.174793
4007135,0.160829


In [6]:
print('Sample lr hr Graphs')
print('-'*50)

source_vectorized = np.genfromtxt("lr_train.csv", delimiter=",", skip_header=1)
source_mat_all = [MatrixVectorizer.anti_vectorize(A, 160) for A in source_vectorized]

source_mat_all = [torch.tensor(x, dtype=torch.float) for x in source_mat_all]
pyg_partial = partial(create_pyg_graph, node_feature_init="adj", node_feat_dim=160)

source_pyg_all = [pyg_partial(x, 160) for x in source_mat_all]
source_data = [{'pyg': source_pyg, 'mat': source_mat} for source_pyg, source_mat in zip(source_pyg_all, source_mat_all)]


tar_vectorized = np.genfromtxt("hr_train.csv", delimiter=",", skip_header=1)
tar_mat_all = [MatrixVectorizer.anti_vectorize(A, 268) for A in tar_vectorized]

tar_mat_all = [torch.tensor(x, dtype=torch.float) for x in tar_mat_all]
pyg_partial = partial(create_pyg_graph, node_feature_init="adj", node_feat_dim=268)

tar_pyg_all = [pyg_partial(x, 268) for x in tar_mat_all]
tar_data = [{'pyg': tar_pyg, 'mat': tar_mat} for tar_pyg, tar_mat in zip(tar_pyg_all, tar_mat_all)]

sample_lr, sample_hr = source_data[0], tar_data[0]

sample_lr_vector = sample_lr['mat'].numpy()
sample_hr_vector = sample_hr['mat'].numpy()

# Now, we use the function for LR and HR data with the right flags
sample_lr_matrix = MatrixVectorizer.anti_vectorize(sample_lr_vector, 160, include_diagonal=False)  
sample_hr_matrix = MatrixVectorizer.anti_vectorize(sample_hr_vector, 268, include_diagonal=False) 

import matplotlib.pyplot as plt
import seaborn as sns
# visualize the original graphs of the last batch of the test set
# Plot the heatmaps
plt.figure(figsize=(18, 8))

# LR Data Heatmap
plt.subplot(1, 2, 1)
sns.heatmap(sample_lr_matrix, square=True, cmap='viridis')
plt.title('LR Data Heatmap (160x160)')

# HR Data Heatmap
plt.subplot(1, 2, 2)
sns.heatmap(sample_hr_matrix, square=True, cmap='viridis')
plt.title('Ground Truth HR Data Heatmap (268x268)')

plt.show()

print('Generated hr Graph')
print('-'*50)
model.eval()
with torch.no_grad():
    pred_sample_lr = sample_lr.to("cuda")
    model_pred, model_target = model(source_g, None)
    pred_m = revert_dual(model_pred, 268)    # (n_t, n_t)
    pred_m = pred_m.cpu().numpy()

    pred_hr_matrix = MatrixVectorizer.anti_vectorize(pred_m, 268, include_diagonal=False) 
    
    plt.figure(figsize=(9, 8))
    sns.heatmap(pred_hr_matrix, square=True, cmap='viridis')
    plt.title('Predicted HR Data Heatmap (268x268)')

    plt.show()

mse = np.mean((pred_hr_matrix - sample_hr_matrix) ** 2)
print("Mean Squared Error:", mse)

Sample lr hr Graphs
--------------------------------------------------


ValueError: setting an array element with a sequence.